In [1]:
%run -i main.py --dataroot data/mnist_26k.npz --model ConvNet --nf 16 32 --nc 1  train --epochs 10 --lr 1e-3 --batch-size 64 --solver Momentum

Namespace(batch_size=64, dataroot='data/mnist_26k.npz', dry_run=False, epochs=5, image_size=28, log_interval=100, lr=0.01, manual_seed=0, model='ConvNet', n_class=10, nc=1, net='', nf=[16, 32], outf='', solver='Momentum', subcommand='train')
(main) Sequential(
  (0) Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1, bias=True)
  (1) BatchNorm2d(num_features=16, eps=1e-05, momentum=0.1, affine=True)
  (2) RELU()
  (3) Maxpool2d(kernel_size=2, stride=2, padding=0)
  (4) Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1, bias=True)
  (5) BatchNorm2d(num_features=32, eps=1e-05, momentum=0.1, affine=True)
  (6) RELU()
  (7) Maxpool2d(kernel_size=2, stride=2, padding=0)
  (8) Flatten()
  (9) Dropout(p=0.3)
  (10) Linear(in_features=1568, out_features=10)
)
==============================Training Start==============================
000000000: #### 0 Epochs: Val Loss = 2.330e+00, Accuracy = 7.40%
000000100: Training Loss = 1.162e+00, Accuracy = 6